In [ ]:
from surprise import KNNBasic, KNNWithMeans, KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import NMF
import pandas as pd

from surprise import SVD
from surprise import evaluate, print_perf
from surprise.model_selection import cross_validate
import numpy as np
import matplotlib.pyplot as plt

#------------------------load data from a file
file_path = "../input/ratings.csv"
ratings = pd.read_csv(file_path, sep=",")
reader = Reader()
ratings.drop("timestamp", axis=1, inplace=True)
data = Dataset.load_from_df(ratings, reader=reader)

#########---------------SVD
# print('')
# print('---------------SVD result-------------')

nmf_rmse = []
svd_rmse = []
# ks = [10, 20, 50, 100, 200]
i = 0
regs = [0.01, 0.1, 1, 10]

while i < 4:
    algo = SVD(n_factors=10, reg_all=regs[i], lr_all=0.005)
    perf = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    rmse = np.sum(perf['test_rmse'])/perf['test_rmse'].size
    svd_rmse.append(rmse)
    
    algo = NMF(n_factors=25, reg_pu=regs[i], reg_qi=regs[i])
    perf = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    rmse = np.sum(perf['test_rmse'])/perf['test_rmse'].size
    nmf_rmse.append(rmse)
    i += 1

plt.plot(nmf_rmse, regs, 'r')
plt.plot(svd_rmse, regs, 'b')
plt.xlabel("RMSE")
plt.ylabel("Regularization Parameters")
plt.show()



















